In [45]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Concatenate, Lambda,RepeatVector
from keras.optimizers import Adam

# Load your wind speed data
# Replace 'your_data.csv' with the actual path to your dataset
df = pd.read_csv('apple_stocks_with_sentiment10.csv', parse_dates=['Date'])
data = df[['Close']]

In [60]:
class NBeatsBlock(tf.keras.layers.Layer):
  def __init__(self, # the constructor takes all the hyperparameters for the layer
               input_size: int,
               theta_size: int,
               horizon: int,
               n_neurons: int,
               n_layers: int,
               **kwargs): # the **kwargs argument takes care of all of the arguments for the parent class (input_shape, trainable, name)
    super().__init__(**kwargs)
    self.input_size = input_size
    self.theta_size = theta_size
    self.horizon = horizon
    self.n_neurons = n_neurons
    self.n_layers = n_layers

    # Block contains stack of 4 fully connected layers each has ReLU activation
    self.hidden = [tf.keras.layers.Dense(n_neurons, activation="relu") for _ in range(n_layers)]
    # Output of block is a theta layer with linear activation
    self.theta_layer = tf.keras.layers.Dense(theta_size, activation="linear", name="theta")

  def call(self, inputs): # the call method is what runs when the layer is called 
    x = inputs 
    for layer in self.hidden: # pass inputs through each hidden layer 
      x = layer(x)
    theta = self.theta_layer(x) 
    # Output the backcast and forecast from theta
    backcast, forecast = theta[:, :self.input_size], theta[:, -self.horizon:]
    return backcast, forecast

In [61]:
bitcoin_prices=data.copy()

HORIZON = 1 # how far to predict forward
WINDOW_SIZE = 20 # how far to lookback

# Add windowed columns
bitcoin_prices_nbeats = bitcoin_prices.copy()
for i in range(WINDOW_SIZE):
  bitcoin_prices_nbeats[f"Price+{i+1}"] = bitcoin_prices_nbeats["Close"].shift(periods=i+1)
bitcoin_prices_nbeats.dropna().head()

# Make features and labels
X = bitcoin_prices_nbeats.dropna().drop("Close", axis=1)
y = bitcoin_prices_nbeats.dropna()["Close"]

# Make train and test sets
split_size = int(len(X) * 0.8)
X_train, y_train = X[:split_size], y[:split_size]
X_test, y_test = X[split_size:], y[split_size:]
len(X_train), len(y_train), len(X_test), len(y_test)

(1596, 1596, 400, 400)

In [62]:
# 1. Turn train and test arrays into tensor Datasets
train_features_dataset = tf.data.Dataset.from_tensor_slices(X_train)
train_labels_dataset = tf.data.Dataset.from_tensor_slices(y_train)

test_features_dataset = tf.data.Dataset.from_tensor_slices(X_test)
test_labels_dataset = tf.data.Dataset.from_tensor_slices(y_test)

# 2. Combine features & labels
train_dataset = tf.data.Dataset.zip((train_features_dataset, train_labels_dataset))
test_dataset = tf.data.Dataset.zip((test_features_dataset, test_labels_dataset))

# 3. Batch and prefetch for optimal performance
BATCH_SIZE = 1024 # taken from Appendix D in N-BEATS paper
train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

train_dataset, test_dataset

(<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 20), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.float64, name=None))>,
 <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 20), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.float64, name=None))>)

In [63]:
# Values from N-BEATS paper Figure 1 and Table 18/Appendix D
N_EPOCHS = 500 # called "Iterations" in Table 18
N_NEURONS = 5 # called "Width" in Table 18
N_LAYERS = 5
N_STACKS = 5

INPUT_SIZE = WINDOW_SIZE * HORIZON # called "Lookback" in Table 18
THETA_SIZE = INPUT_SIZE + HORIZON

INPUT_SIZE, THETA_SIZE

(20, 21)

In [64]:
from tensorflow.keras import layers

# 1. Setup N-BEATS Block layer
nbeats_block_layer = NBeatsBlock(input_size=INPUT_SIZE,
                                 theta_size=THETA_SIZE,
                                 horizon=HORIZON,
                                 n_neurons=N_NEURONS,
                                 n_layers=N_LAYERS,
                                 name="InitialBlock")

# 2. Create input to stacks
stack_input = layers.Input(shape=(INPUT_SIZE), name="stack_input")

# 3. Create initial backcast and forecast input (backwards predictions are referred to as residuals in the paper)
backcast, forecast = nbeats_block_layer(stack_input)
# Add in subtraction residual link, thank you to: https://github.com/mrdbourke/tensorflow-deep-learning/discussions/174 
residuals = layers.subtract([stack_input, backcast], name=f"subtract_00") 

# 4. Create stacks of blocks
for i, _ in enumerate(range(N_STACKS-1)): # first stack is already creted in (3)

  # 5. Use the NBeatsBlock to calculate the backcast as well as block forecast
  backcast, block_forecast = NBeatsBlock(
      input_size=INPUT_SIZE,
      theta_size=THETA_SIZE,
      horizon=HORIZON,
      n_neurons=N_NEURONS,
      n_layers=N_LAYERS,
      name=f"NBeatsBlock_{i}"
  )(residuals) # pass it in residuals (the backcast)

  # 6. Create the double residual stacking
  residuals = layers.subtract([residuals, backcast], name=f"subtract_{i}") 
  forecast = layers.add([forecast, block_forecast], name=f"add_{i}")

# 7. Put the stack model together
model_7 = tf.keras.Model(inputs=stack_input, 
                         outputs=forecast, 
                         name="model_7_N-BEATS")

# 8. Compile with MAE loss and Adam optimizer
model_7.compile(loss="mae",
                optimizer=tf.keras.optimizers.Adam(0.001),
                metrics=["mae", "mse"])

# 9. Fit the model with EarlyStopping and ReduceLROnPlateau callbacks
model_7.fit(train_dataset,
            epochs=N_EPOCHS,
            validation_data=test_dataset,
            verbose=2, # prevent large amounts of training outputs
            # callbacks=[create_model_checkpoint(model_name=stack_model.name)] # saving model every epoch consumes far too much time
            callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=20, restore_best_weights=True),
                      tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=10, verbose=2)])

Epoch 1/500
2/2 - 9s - loss: 21.1587 - mae: 21.1587 - mse: 507.3382 - val_loss: 43.7582 - val_mae: 43.7582 - val_mse: 2002.5001 - lr: 0.0010 - 9s/epoch - 5s/step
Epoch 2/500
2/2 - 0s - loss: 18.9262 - mae: 18.9262 - mse: 405.7493 - val_loss: 38.7314 - val_mae: 38.7314 - val_mse: 1569.9410 - lr: 0.0010 - 60ms/epoch - 30ms/step
Epoch 3/500
2/2 - 0s - loss: 16.6470 - mae: 16.6470 - mse: 313.6391 - val_loss: 33.6323 - val_mae: 33.6323 - val_mse: 1185.1610 - lr: 0.0010 - 51ms/epoch - 26ms/step
Epoch 4/500
2/2 - 0s - loss: 14.3478 - mae: 14.3478 - mse: 232.8394 - val_loss: 28.5819 - val_mae: 28.5819 - val_mse: 857.7292 - lr: 0.0010 - 55ms/epoch - 28ms/step
Epoch 5/500
2/2 - 0s - loss: 12.0487 - mae: 12.0487 - mse: 164.0735 - val_loss: 23.1972 - val_mae: 23.1972 - val_mse: 567.1703 - lr: 0.0010 - 58ms/epoch - 29ms/step
Epoch 6/500
2/2 - 0s - loss: 9.5806 - mae: 9.5806 - mse: 103.6266 - val_loss: 17.3895 - val_mae: 17.3895 - val_mse: 321.6155 - lr: 0.0010 - 61ms/epoch - 30ms/step
Epoch 7/500
2

Epoch 52/500
2/2 - 0s - loss: 0.8667 - mae: 0.8667 - mse: 1.3735 - val_loss: 2.0448 - val_mae: 2.0448 - val_mse: 7.9606 - lr: 1.0000e-04 - 46ms/epoch - 23ms/step
Epoch 53/500
2/2 - 0s - loss: 0.8664 - mae: 0.8664 - mse: 1.3722 - val_loss: 2.0391 - val_mae: 2.0391 - val_mse: 7.9352 - lr: 1.0000e-04 - 44ms/epoch - 22ms/step
Epoch 54/500
2/2 - 0s - loss: 0.8662 - mae: 0.8662 - mse: 1.3711 - val_loss: 2.0337 - val_mae: 2.0337 - val_mse: 7.9112 - lr: 1.0000e-04 - 43ms/epoch - 22ms/step
Epoch 55/500
2/2 - 0s - loss: 0.8661 - mae: 0.8661 - mse: 1.3703 - val_loss: 2.0287 - val_mae: 2.0287 - val_mse: 7.8903 - lr: 1.0000e-04 - 47ms/epoch - 24ms/step
Epoch 56/500
2/2 - 0s - loss: 0.8661 - mae: 0.8661 - mse: 1.3699 - val_loss: 2.0246 - val_mae: 2.0246 - val_mse: 7.8728 - lr: 1.0000e-04 - 50ms/epoch - 25ms/step
Epoch 57/500
2/2 - 0s - loss: 0.8661 - mae: 0.8661 - mse: 1.3697 - val_loss: 2.0215 - val_mae: 2.0215 - val_mse: 7.8597 - lr: 1.0000e-04 - 49ms/epoch - 25ms/step
Epoch 58/500

Epoch 58: Redu

In [70]:
def make_preds(model, input_data):
  """
  Uses model to make predictions on input_data.

  Parameters
  ----------
  model: trained model 
  input_data: windowed input data (same kind of data model was trained on)

  Returns model predictions on input_data.
  """
  forecast = model.predict(input_data)
  return tf.squeeze(forecast) # return 1D array of predictions

In [71]:
# Make predictions with N-BEATS model
model_7_preds = make_preds(model_7, test_dataset)
model_7_preds[:10]

1/1 [==============================] - 0s 16ms/step


<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([47.224   , 47.44925 , 47.908127, 48.40263 , 48.415485, 48.770153,
       48.87805 , 49.210464, 49.203667, 48.921753], dtype=float32)>

In [72]:
# MASE implementation
def mean_absolute_scaled_error(y_true, y_pred):
    mae = tf.reduce_mean(tf.abs(y_true-y_pred))
    
    # Find MAE of naive forecast
    mae_naive_no_season = tf.reduce_mean(tf.abs(y_true[1:]-y_true[:-1]))
    
    return mae / mae_naive_no_season

In [73]:
def evaluate_preds(y_true, y_pred):
  # Make sure float32 (for metric calculations)
  y_true = tf.cast(y_true, dtype=tf.float32)
  y_pred = tf.cast(y_pred, dtype=tf.float32)

  # Calculate various metrics
  mae = tf.keras.metrics.mean_absolute_error(y_true, y_pred)
  mse = tf.keras.metrics.mean_squared_error(y_true, y_pred) # puts and emphasis on outliers (all errors get squared)
  rmse = tf.sqrt(mse)
  mape = tf.keras.metrics.mean_absolute_percentage_error(y_true, y_pred)
  mase = mean_absolute_scaled_error(y_true, y_pred)
  
  return {"mae": mae.numpy(),
          "mse": mse.numpy(),
          "rmse": rmse.numpy(),
          "mape": mape.numpy(),
          "mase": mase.numpy()}

In [74]:
# Evaluate N-BEATS model predictions
model_7_results = evaluate_preds(y_true=y_test,
                                 y_pred=model_7_preds)
model_7_results

{'mae': 1.9829636,
 'mse': 7.803687,
 'rmse': 2.793508,
 'mape': 3.4970407,
 'mase': 2.099822}